In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
data = pd.read_csv(r"./data_2.csv")
data=data.drop(['user_id'], axis = 1)
data=data.drop(['jd_no'], axis = 1)
data=data.drop(['Unnamed: 0'], axis = 1)
data.isnull().sum()

delivered                     0
satisfied                     0
is_travel                     0
min_years                     0
period                        0
birthday                      0
start_work_date               0
edu_dif                       0
max_desire_salary_dif         0
min_desire_salary_dif         0
max_cur_salary_dif            0
min_cur_salary_dif            0
city_match                    0
Unstructured_similarity    1051
String_similarity             0
dtype: int64

In [2]:
# 去除空值
data = data.dropna(axis = 0,subset =['Unstructured_similarity'])
data.isnull().sum()

delivered                  0
satisfied                  0
is_travel                  0
min_years                  0
period                     0
birthday                   0
start_work_date            0
edu_dif                    0
max_desire_salary_dif      0
min_desire_salary_dif      0
max_cur_salary_dif         0
min_cur_salary_dif         0
city_match                 0
Unstructured_similarity    0
String_similarity          0
dtype: int64

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132013 entries, 0 to 133063
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   delivered                132013 non-null  int64  
 1   satisfied                132013 non-null  int64  
 2   is_travel                132013 non-null  int64  
 3   min_years                132013 non-null  float64
 4   period                   132013 non-null  float64
 5   birthday                 132013 non-null  float64
 6   start_work_date          132013 non-null  float64
 7   edu_dif                  132013 non-null  float64
 8   max_desire_salary_dif    132013 non-null  float64
 9   min_desire_salary_dif    132013 non-null  float64
 10  max_cur_salary_dif       132013 non-null  float64
 11  min_cur_salary_dif       132013 non-null  float64
 12  city_match               132013 non-null  int64  
 13  Unstructured_similarity  132013 non-null  float64
 14  Stri

In [5]:
# Splitting into train and test sets
X = data.drop(['delivered','satisfied'], axis = 1)
y_d = data['delivered']
y_s = data['satisfied']

In [6]:
X_train, X_test, y_strain, y_stest = train_test_split(X, y_s, test_size = 0.2, random_state = 628)
print(len(X_train), len(X_test))

105610 26403


In [7]:
X_train, X_test, y_dtrain, y_dtest = train_test_split(X, y_d, test_size = 0.2, random_state = 628)
print(len(X_train), len(X_test))

105610 26403


## logistics

In [8]:
#satisfied
from sklearn.linear_model import LogisticRegression

# Fitting a logistic regression model with default parameters
logreg = LogisticRegression()
logreg.fit(X_train, y_strain)

# print the estimated coefficient for each variable
coef = pd.DataFrame({'variable':list(X_train.columns),'coef': logreg.coef_[0]})
coef.sort_values('coef',ascending=False)

,variable,coef
2,period,0.054714
5,edu_dif,0.054424
6,max_desire_salary_dif,0.039027
4,start_work_date,0.033449
12,String_similarity,-0.002905
3,birthday,-0.005227
9,min_cur_salary_dif,-0.005490
8,max_cur_salary_dif,-0.023368
7,min_desire_salary_dif,-0.065719
10,city_match,-0.137869


In [9]:
# Prediction 
y_shat_test = logreg.predict(X_test)# predict()预测为0/1

# Precision, Recall, F1-score
print("Logistic regression score for test set:")

precision = precision_score(y_stest, y_shat_test)
recall = recall_score(y_stest, y_shat_test)
f1 = f1_score(y_stest, y_shat_test)

print("Precision: {:.3f}".format(precision), "Recall: {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

# AUC
y_shat_test = logreg.predict_proba(X_test)# predict_proba预测为1的概率
print("AUC score: {:.3f}".format(roc_auc_score(y_stest, y_shat_test[:,1])))

Logistic regression score for test set:
Precision: 0.000 Recall: 0.000
F1 score: 0.000
AUC score: 0.593


D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
#样本均衡
y_shat_test = logreg.predict(X_test)# predict()预测为0/1
logreg = LogisticRegression(class_weight = 'balanced')
logreg.fit(X_train, y_strain)

# Prediction
y_shat_test = logreg.predict(X_test)

# evaluation
# Precision, Recall, F1-score
print("Logistic regression score for test set:")
precision = precision_score(y_stest, y_shat_test)
recall = recall_score(y_stest, y_shat_test)
f1 = f1_score(y_stest, y_shat_test)

print("Precision: {:.3f}".format(precision), "Recall: {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

# AUC
y_shat_test = logreg.predict_proba(X_test)
print("AUC score: {:.3f}".format(roc_auc_score(y_stest, y_shat_test[:,1])))

Logistic regression score for test set:
Precision: 0.266 Recall: 0.607
F1 score: 0.370
AUC score: 0.592


In [11]:
#delivered
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_dtrain, y_dtest = train_test_split(X, y_d, test_size = 0.2, random_state = 628)
# Fitting a logistic regression model with default parameters
logreg = LogisticRegression()
logreg.fit(X_train, y_dtrain)

# print the estimated coefficient for each variable
coef = pd.DataFrame({'variable':list(X_train.columns),'coef': logreg.coef_[0]})
coef.sort_values('coef',ascending=False)

,variable,coef
11,Unstructured_similarity,0.186826
3,birthday,0.123207
1,min_years,0.044971
6,max_desire_salary_dif,0.032343
9,min_cur_salary_dif,0.031390
5,edu_dif,0.017615
7,min_desire_salary_dif,0.009037
2,period,0.007018
4,start_work_date,0.000950
12,String_similarity,-0.000472


In [12]:
# Prediction 
y_dhat_test = logreg.predict(X_test)# predict()预测为0/1

# Precision, Recall, F1-score
print("Logistic regression score for test set:")

precision = precision_score(y_dtest, y_dhat_test)
recall = recall_score(y_dtest, y_dhat_test)
f1 = f1_score(y_dtest, y_dhat_test)

print("Precision: {:.3f}".format(precision), "Recall: {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

# AUC
y_dhat_test = logreg.predict_proba(X_test)# predict_proba预测为1的概率
print("AUC score: {:.3f}".format(roc_auc_score(y_dtest, y_dhat_test[:,1])))

Logistic regression score for test set:
Precision: 0.559 Recall: 0.238
F1 score: 0.334
AUC score: 0.556


In [13]:
y_dhat_test = logreg.predict(X_test)# predict()预测为0/1
logreg = LogisticRegression(class_weight = 'balanced')
logreg.fit(X_train, y_dtrain)

# Prediction
y_dhat_test = logreg.predict(X_test)

# evaluation
# Precision, Recall, F1-score
print("Logistic regression score for test set:")
precision = precision_score(y_dtest, y_dhat_test)
recall = recall_score(y_dtest, y_dhat_test)
f1 = f1_score(y_dtest, y_dhat_test)

print("Precision: {:.3f}".format(precision), "Recall: {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

# AUC
y_dhat_test = logreg.predict_proba(X_test)
print("AUC score: {:.3f}".format(roc_auc_score(y_dtest, y_dhat_test[:,1])))

Logistic regression score for test set:
Precision: 0.518 Recall: 0.478
F1 score: 0.497
AUC score: 0.556


## random forest

In [14]:
# random forest
#s
# Fitting a Random Forest model with parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [15]:
#s
param_grid = {
    'n_estimators':[100,150,200,250,300],
    'max_depth': [5,10,15,20,25,30]
}
gs = GridSearchCV(RandomForestClassifier(), param_grid, scoring='roc_auc')
gs.fit(X_train,y_strain)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15, 20, 25, 30],
                         'n_estimators': [100, 150, 200, 250, 300]},
             scoring='roc_auc')

In [16]:
#输出在验证集上的最佳表现和对应参数
print('最好分数：', gs.best_score_)
print('对应参数：', gs.best_params_)

最好分数： 0.6677373618711406
对应参数： {'max_depth': 15, 'n_estimators': 300}


In [18]:
clf_s_r = RandomForestClassifier(n_estimators = 300, max_depth = 15)
clf_s_r.fit(X_train, y_strain)

RandomForestClassifier(max_depth=15, n_estimators=300)

In [19]:
# Prediction
y_shat_test = clf_s_r.predict(X_test)

In [20]:
# Precision, Recall, F1-score
print("Random Forest Classifier score for test set:")

precision = precision_score(y_stest, y_shat_test)
recall = recall_score(y_stest, y_shat_test)
f1 = f1_score(y_stest, y_shat_test)

print("Precision: {:.3f}".format(precision), "Recall: {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

# AUC
y_shat_test = clf_s_r.predict_proba(X_test)# predict_proba预测为1的概率
print("AUC score: {:.3f}".format(roc_auc_score(y_stest, y_shat_test[:,1])))

Random Forest Classifier score for test set:
Precision: 0.628 Recall: 0.061
F1 score: 0.111
AUC score: 0.676


In [21]:
# compute the importance of a feature
importances = pd.DataFrame({'variable':list(X_train.columns),'importances': clf_s_r.feature_importances_})
importances.sort_values('importances',ascending =False)

,variable,importances
11,Unstructured_similarity,0.165808
2,period,0.143167
1,min_years,0.096393
12,String_similarity,0.093852
9,min_cur_salary_dif,0.079382
7,min_desire_salary_dif,0.076208
4,start_work_date,0.074041
6,max_desire_salary_dif,0.071265
8,max_cur_salary_dif,0.070990
3,birthday,0.047728


In [22]:
#d
param_grid = {
    'n_estimators':[100,150,200,250,300],
    'max_depth': [5,10,15,20,25,30]
}
gs = GridSearchCV(RandomForestClassifier(), param_grid, scoring='roc_auc')
gs.fit(X_train,y_dtrain)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15, 20, 25, 30],
                         'n_estimators': [100, 150, 200, 250, 300]},
             scoring='roc_auc')

In [23]:
#输出在验证集上的最佳表现和对应参数
print('最好分数：', gs.best_score_)
print('对应参数：', gs.best_params_)

最好分数： 0.6022309234292976
对应参数： {'max_depth': 15, 'n_estimators': 200}


In [24]:
clf = RandomForestClassifier(n_estimators = 200, max_depth = 15)
clf.fit(X_train, y_dtrain)

RandomForestClassifier(max_depth=15, n_estimators=200)

In [25]:
# Prediction
y_dhat_test = clf.predict(X_test)

In [26]:
# Precision, Recall, F1-score
print("Random Forest Classifier score for test set:")

precision = precision_score(y_dtest, y_dhat_test)
recall = recall_score(y_dtest, y_dhat_test)
f1 = f1_score(y_dtest, y_dhat_test)

print("Precision: {:.3f}".format(precision), "Recall: {:.3f}".format(recall))
print("F1 score: {:.3f}".format(f1))

# AUC
y_dhat_test = clf.predict_proba(X_test)# predict_proba预测为1的概率
print("AUC score: {:.3f}".format(roc_auc_score(y_dtest, y_dhat_test[:,1])))

Random Forest Classifier score for test set:
Precision: 0.598 Recall: 0.315
F1 score: 0.413
AUC score: 0.602


In [27]:
# compute the importance of a feature
importances = pd.DataFrame({'variable':list(X_train.columns),'importances': clf.feature_importances_})
importances.sort_values('importances',ascending =False)

,variable,importances
11,Unstructured_similarity,0.176159
2,period,0.106582
12,String_similarity,0.100890
9,min_cur_salary_dif,0.092243
4,start_work_date,0.090338
7,min_desire_salary_dif,0.087404
6,max_desire_salary_dif,0.082586
8,max_cur_salary_dif,0.076583
3,birthday,0.060009
1,min_years,0.045205


## xgboost


In [28]:
#satisfied
param_grid = {
    'n_estimators':[100,125,150,175,200,225,250],
    'max_depth': [2,3,4,5,6,8,10,15]
}
gs = GridSearchCV(XGBClassifier(), param_grid, scoring='roc_auc')
gs.fit(X_train,y_strain)

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 8, 10, 15],
                         'n_estimators': [100, 125, 150, 175, 200, 225, 250]},
             scoring='roc_auc')

In [29]:
#输出在验证集上的最佳表现和对应参数
print('最好分数：', gs.best_score_)
print('对应参数：', gs.best_params_)

最好分数： 0.6678651671880961
对应参数： {'max_depth': 3, 'n_estimators': 250}


In [30]:
#根据最优参数重新训练模型和预测
clf = XGBClassifier(max_depth = 3,n_estimators=250) 
clf.fit(X_train, y_strain)
y_shat_test = clf.predict(X_test)
print("Precision: {:.3f}".format(precision_score(y_stest, y_shat_test)), "Recall: {:.3f}".format(recall_score(y_stest, y_shat_test)))
print("F1 score: {:.3f}".format(f1_score(y_stest, y_shat_test)))
# AUC
y_shat_test = clf.predict_proba(X_test)
print("AUC score: {:.3f}".format(roc_auc_score(y_stest, y_shat_test[:,1])))

Precision: 0.553 Recall: 0.064
F1 score: 0.114
AUC score: 0.674


In [31]:
# compute the importance of a feature
importances = pd.DataFrame({'variable':list(X_train.columns),'importances': clf.feature_importances_})
importances.sort_values('importances',ascending =False)

,variable,importances
1,min_years,0.387523
0,is_travel,0.147642
2,period,0.098494
10,city_match,0.060456
5,edu_dif,0.050582
8,max_cur_salary_dif,0.043909
11,Unstructured_similarity,0.037327
3,birthday,0.032330
9,min_cur_salary_dif,0.030663
6,max_desire_salary_dif,0.030137


In [32]:
#delivered
param_grid = {
    'n_estimators':[100,125,150,175,200,225,250],
    'max_depth': [2,2,4,5,6,8,10,15,20]
}
gs = GridSearchCV(XGBClassifier(), param_grid, scoring='roc_auc')
gs.fit(X_train,y_dtrain)

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'max_depth': [2, 2, 4, 5, 6, 8, 10, 15, 20],
                         'n_estimators': [100, 125, 150, 175, 200, 225, 250]},
             scoring='roc_auc')

In [33]:
print('最好分数：', gs.best_score_)
print('对应参数：', gs.best_params_)

最好分数： 0.600910162128786
对应参数： {'max_depth': 4, 'n_estimators': 225}


In [34]:
clf_d_xg = XGBClassifier(max_depth = 4,n_estimators=225) 
clf_d_xg.fit(X_train, y_dtrain)
y_dhat_test = clf_d_xg.predict(X_test)
print("Precision: {:.3f}".format(precision_score(y_dtest, y_dhat_test)), "Recall: {:.3f}".format(recall_score(y_dtest, y_dhat_test)))
print("F1 score: {:.3f}".format(f1_score(y_dtest, y_dhat_test)))
# AUC
y_dhat_test = clf_d_xg.predict_proba(X_test)
print("AUC score: {:.3f}".format(roc_auc_score(y_dtest, y_dhat_test[:,1])))

Precision: 0.580 Recall: 0.371
F1 score: 0.452
AUC score: 0.601


In [35]:
# compute the importance of a feature
importances = pd.DataFrame({'variable':list(X_train.columns),'importances': clf_d_xg.feature_importances_})
importances.sort_values('importances',ascending =False)

,variable,importances
1,min_years,0.123290
3,birthday,0.115563
10,city_match,0.085093
4,start_work_date,0.084967
6,max_desire_salary_dif,0.075476
11,Unstructured_similarity,0.072174
12,String_similarity,0.071365
2,period,0.068775
5,edu_dif,0.066250
9,min_cur_salary_dif,0.065639


F1希望训练出一个不放过任何一种可能的模型比如欺诈检测、癌症检测，漏诊是不负责的，宁可错杀一千，也不放过一个  
AUC希望训练出一个尽量不误报的模型，比如推荐算法，推荐错误用户会非常反感  
XGBoost和随机森林模型结果的AUC值均大于逻辑回归

In [36]:
y_shat = clf_s_r.predict_proba(X)
y_dhat = clf_d_xg.predict_proba(X)

In [37]:
S = y_s
D = y_d
S_hat = y_shat[:,1]
D_hat = y_dhat[:,1]

In [38]:
import numpy as np
from sklearn import metrics

w_list = [m*0.01 for m in range(100)]
rmse=1
best_w=0
for w in w_list:
    y_hat = w * S_hat + (1-w)* D_hat
    y = w * S + (1-w)* D
    res_rmse = np.sqrt(metrics.mean_squared_error(y, y_hat))
    #print(res_rmse)
    if res_rmse < rmse:
        rmse = res_rmse
        best_w = w
        score = y_hat

In [39]:
best_w

0.8200000000000001

In [40]:
rmse

0.34994941627448434

In [41]:
score

array([0.30306826, 0.50126723, 0.1873529 , ..., 0.24579299, 0.35629932,
       0.55950689])

In [42]:
len(score)

132013

In [43]:
data_final = pd.read_csv('data.csv')
data_final = data_final.dropna(axis = 0,subset =['Unstructured_similarity'])
data_final.shape

(132013, 17)

In [44]:
data_final.loc[:,'score']=score

In [45]:
data_final.to_csv('data_final.csv',encoding='utf_8_sig')

In [46]:
import joblib
joblib.dump(clf_s_r,'save_clf_s_r.pkl')
joblib.dump(clf_d_xg,'save_clf_d_xg.pkl')

['save_clf_d_xg.pkl']